In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime as dt
pd.set_option('display.max_rows', None)
import tabula
from tabula.io import read_pdf
import os
import sys

In [2]:
# # rebecca

# path = r'C:\Users\rebecca.lee\OneDrive - GC International AG\Rebecca Lee\orderListData\input_data\\'

# path_output = r'C:\Users\rebecca.lee\OneDrive - GC International AG\Rebecca Lee\orderListData\output_data\\'

In [3]:
# stephen

path_output = r'C:\Users\stephenl\OneDrive - GC International AG\ML\Trial\data_output\\'

path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\Trial\data_input\\'

# Functions

In [4]:
# separate product code and product packaging
def splitnJoin(x):
    remove0 = lambda x: x.split(' ')[0]
    remove1 = lambda x: x.split(' ')[1:]
    res1 = remove0(x)
    _=remove1(x)
    res2 = ' '.join(_)
    return res1, res2

nandate = lambda x: dt.now() if x is null() else x

# function to extract PO number
def extractPo(df_):
    po = df_[0]
    po=po.fillna('nann')
    po_total = []
    for p in po.iloc[:,3]:
        if re.search(r'^[A-Z]\d{3}\s',p) or re.search(r'^[A-Z]\d{3}',p) or re.search(r'^[0-9]*$',p):
            p=p.replace('/','_').replace('-','_')
            po_total.append(p)
    if len(po_total)>0:
        return ''.join(po_total)
    else:
        return 'no po number available'



# extract expiry month and year
def toMthYr(s):
    if re.search('\-[A-Z]\d{4}$',s):
        return dt.strptime(s[-4:][0:2]+'-'+s[-4:][2:4],"%m-%y")
    else:
        return dt.now()


# calculate date to expiry
def month_diff(x,y):
    end = x.dt.to_period('M').view(dtype='int64')
    start = y.dt.to_period('M').view(dtype='int64')
    return end - start

def otherCharges(df_):
    df=df_[-1]
    
    ship_instr = ship(df_)
    df_new = pd.DataFrame(columns=['material_no','material_desc','batch','qty','unit_listprice','unit_discprice',\
                                        'list_pricetotal','disc_pricetotal','po_num','exp_date','num_months'])
    
    df_new.loc[0,'exp_date']='Shipping Instructions'
    df_new.loc[0,'num_months']=ship_instr
    if 'Freight Charge' in df.iloc[:,0].tolist():
        df_new.loc[1,'num_months']=df[df.iloc[:,0]=='Freight Charge'].Discount.iloc[0]
        df_new.loc[1,'exp_date']='Freight Charge'
    if 'Insurance Charge' in df.iloc[:,0].tolist():
        df_new.loc[2,'num_months']=df[df.iloc[:,0]=='Insurance Charge'].Discount.iloc[0]
        df_new.loc[2,'exp_date']='Insurance Charge'
    if 'Packing Charge' in df.iloc[:,0].tolist():
        df_new.loc[3,'num_months']=df[df.iloc[:,0]=='Packing Charge'].Discount.iloc[0]
        df_new.loc[3,'exp_date']='Packing Charge'
    if 'Total Excluding GST' in df.iloc[:,4].tolist():
        df_new.loc[4,'num_months']=df[df.iloc[:,4]=='Total Excluding GST'].Discount.iloc[0]
        df_new.loc[4,'exp_date']='Total Excluding GST'
    if 'Other Charges' in df.iloc[:,4].tolist():
        df_new.loc[5,'num_months']=df[df.iloc[:,4]=='Other Charges'].Discount.iloc[0]
        df_new.loc[5,'exp_date']='Other Charges'
    if 'Add GST @ 0%' in df.iloc[:,4].tolist():
        df_new.loc[6,'num_months']=df[df.iloc[:,4]=='Add GST @ 0%'].Discount.iloc[0]
        df_new.loc[6,'exp_date']='Add GST @ 0%'
    if 'Add GST @ 7%' in df.iloc[:,4].tolist():
        df_new.loc[6,'num_months']=df[df.iloc[:,4]=='Add GST @ 7%'].Discount.iloc[0]
        df_new.loc[6,'exp_date']='Add GST @ 7%'
    if 'Invoice Total Including GST' in df.iloc[:,4].tolist():
        df_new.loc[7,'num_months']=df[df.iloc[:,4]=='Invoice Total Including GST'].Discount.iloc[0]
        df_new.loc[7,'exp_date']='Invoice Total Including GST'
        
    for idx, i in zip(df_new.index, df_new.num_months):
        try:
            df_new.loc[idx,'num_months']=float(i)
        except:
            continue
        
    return df_new


def regsplit(m):
    trim = lambda x: ' '.join(x.split())
    g = re.search('.*\-\d{3,4}[A-Z]',m) or re.search('^\d{6}',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}',m)\
    or re.search('^[A-Z]\d{8}',m) or re.search('.*\-\d{3,4}',m)
    g = g.group()
    g2 = m.replace(g,'')
    return trim(g),trim(g2)

# returns only
def regsplit2(m):
    trim = lambda x: ' '.join(x.split())
    g = re.search('.*\-\d{3,4}[A-Z]',m) or re.search('^.*\d{6}',m) or re.search('^.*[A-Z]{2}\d{2}\-[A-Z]{2}',m)\
    or re.search('^.*[A-Z]\d{8}',m) or re.search('.*\-\d{3,4}',m)
    g = g.group()
    return trim(g)

def check_space(string):
     
    # counter
    count = 0
    
    # loop for search each index
    for i in range(0, len(string)):
         
        # Check each char
        # is blank or not
        if string[i] == " ":
            count += 1
         
    return count

# function to return shipping instructions
def ship(df_):
    si=df_[0]
    si=si.dropna(subset=['Unnamed: 2'])
    if 'Shipping Instructions:' in si['Unnamed: 2'].tolist():
        idx = si[si['Unnamed: 2']=='Shipping Instructions:'].index[0]
        si_list = si.loc[idx+1:,'Unnamed: 2']
        si_list1 = si_list.tolist()
        si_list2 = ','.join(si_list1)
    else:
        si_list2 = 'No Shipping Instructions Available'
    return si_list2


def NPpcodesPI(i): 
    return pi_np[pi_np['material_no']==i]

def NPpcodesOA(i):
    return oa_np[oa_np['material_no']==i]

def PpcodesPI(i): 
    return pi_p[pi_p['material_no']==i]

def PpcodesOA(i):
    return oa_p[oa_p['material_no']==i]

def expiry():
    a = temp_under16
    poNum = a.po_numT.unique()
    numPdts = a.qty.sum()
    numPCodes = len(a.material_no.unique())
    return poNum, numPdts, numPCodes

def numPo():
    a = all_pi_df
    pos = a['customer_purchase_order_#_T'].unique()
    numPos = len(a['customer_purchase_order_#_T'].unique())
    return pos, numPos

# extract 2 columns 
def trimm(i):
    return i[['material_no','po_numT']]

# convert series to list
def listit(seri):
    pc = []
    for w in seri:
        pc.append(w)
    return pc


# # final status output for reconciliation
# def NPstatus():
#     a = NP_status[NP_status.status=='Discrepancies in OA']
#     mat = a.material_no
#     po = a.po_numT.unique()
#     req = (NP_status.status=='Missing code in OA') | (NP_status.status=='Missing code in PI')
#     b = NP_status[req]
#     pc = b.material_no
#     pom = b.po_numT.unique()
#     return listit(po),listit(mat), listit(pc),listit(pom)


# final status output for reconciliation
def NPstatus():
    a = NP_status[NP_status.status=='Discrepancies in OA']
    mat = a.material_no.unique()
    po = a.po_numT.unique()
    req = (NP_status.status=='Missing code in OA') | (NP_status.status=='Missing code in PI')
    b = NP_status[req]
    pc = b.material_no.unique()
    pom = b.po_numT.unique()
    return listit(po),listit(mat), listit(pc),listit(pom)


def Pstatus():
    a = P_status[P_status.status=='Discrepancies in OA'] 
    mat = a.material_no.unique()
    po = a.po_numT.unique()
    req = (P_status.status=='Missing code in OA') | (P_status.status=='Missing code in PI')
    b = P_status[req]
    pc = b.material_no.unique()
    pom = b.po_numT.unique()
    return listit(po),listit(mat), listit(pc),listit(pom)

def overallPO():
    a = all_pi_df
    pc = len(all_pi_df['customer_purchase_order_#'].unique())
    return pc

missingtext = lambda x: 'Missing code in PI' if x=='Discrepancies in OA' else 'Missing code in OA' if x=='Correct items in PI' else \
'NA'

In [5]:
def execute(df_):
    
    # extract customer po & shipping instructions

    po_num = extractPo(df_)

    # create dataframe for packing charge etc
    others = otherCharges(df_)
    
    
    # create dataframes of all relevant rows

    # length of dataframe (raw)
    leng = len(df_)

    # create new dataframe
    temp = pd.DataFrame()

    # concat dataframes with 9 columns which have product codes

    for i in range(0, leng):
        
        if df_[i].shape[1]==9:
            temp=pd.concat([temp,df_[i]])

        elif df_[i].shape[1]>9:
            with open(path_output+f"Alert_{po_num}_.txt", "w") as f:
                f.write(f'OA file with PO: {po_num} has to be fixed before running.')
            print(f"{po_num} has to be fixed before running.")

            continue
        else:
            continue
            


    # reset index
    temp = temp.reset_index(drop=True)
    
    req = (temp['Unnamed: 3']=='EA') | (temp['Unnamed: 3']=='BOX')
    temp=temp[req]
    
    
    # set as string
    temp.iloc[:,0]=temp.iloc[:,0].astype(str)

    # find the last row with product code
    tot=[]
    for idx, m in zip(temp.index,temp.iloc[:,0]):
        
        if re.search('.*\-\d{3,4}\D.*',m) or re.search('.*\-\d{3,4}\s',m) or re.search('.*\-\d{3,4}$',m)\
            or re.search('^\d{6}\s',m) or re.search('^\d{6}\D.*',m) or re.search('^\d{6}$',m)\
            or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\D*',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\s',m)\
        or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}$',m)\
            or re.search('^[A-Z]\d{8}\D.*',m) or re.search('^[A-Z]\d{8}\s.*',m) or re.search('^[A-Z]\d{8}$',m):
            tot.append(idx)
        else:
            continue

    # replace 'nan' with null()
    temp.replace('nan',np.nan, inplace=True)


    # last row with valid product code
    # ffill empty rows for columns 0 and 3

    temp.iloc[:max(tot)+2,0].fillna(method='ffill',inplace=True)
    temp.iloc[:max(tot)+2,3].fillna(method='ffill',inplace=True)

    # convert to strings
    temp.iloc[:,0]=temp.iloc[:,0].astype(str)
    
    # extract row index with correct product code
    idx=[]
    for ind,m in zip(temp.index,temp.iloc[:,0]):
        
        if re.search('.*\-\d{3,4}\D.*',m) or re.search('.*\-\d{3,4}\s',m) or re.search('.*\-\d{3,4}$',m)\
        or re.search('^\d{6}\s',m) or re.search('^\d{6}\D.*',m) or re.search('^\d{6}$',m)\
        or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\D*',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\s',m)\
    or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}$',m)\
        or re.search('^[A-Z]\d{8}\D.*',m) or re.search('^[A-Z]\d{8}\s.*',m) or re.search('^[A-Z]\d{8}$',m):
                idx.append(ind)
        else:
            continue
    
    # extract only rows with correct product code using index
    temp= temp.loc[idx]
    temp=temp.reset_index(drop=True)
    
    # copy to df
    df=temp.copy()

    # create new dataframe to transfer all info over
    new = pd.DataFrame(columns=['material_no','material_desc','batch','qty','unit_listprice','unit_discprice',\
                                'list_pricetotal','disc_pricetotal','po_num']) #-------------------------------------------

    # fill in  new dataframe (df is the old dataframe)
    matno=[]
    matdes=[]
        # if both mat'l no and mat'l description are in column 1 of old dataframe. This will separate them.
    for i,m,d in zip(df.index,df.iloc[:,0],df.iloc[:,1]): # 1st column
        if re.search('.*\-\d{3,4}\D.*',m) or re.search('.*\-\d{3,4}\s',m)\
        or re.search('^\d{6}\s',m) or re.search('^\d{6}\D.*',m)\
        or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\D*',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\s',m)\
        or re.search('^[A-Z]\d{8}\D.*',m) or re.search('^[A-Z]\d{8}\s.*',m):
            res1,res2 = regsplit(m)
            
            new.loc[i,'material_no'] =res1 # 1st column
            new.loc[i,'material_desc'] = res2 # 2nd column
        
        # if both mat'l no and mat'l description are in separate columns of the old dataframe, then no need to separate
        # just assign to the respective columns in the "new" dataframe
        elif re.search('.*\-\d{3,4}$',m) or re.search('^\d{6}$',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}$',m)\
        or re.search('^[A-Z]\d{8}$',m):
            new.loc[i,'material_no'] = m # 1st column
            new.loc[i,'material_desc'] = d # 2nd column

    # fill in batch number
    new.batch = df.iloc[:,3] # 3rd column
    

    df.iloc[:,5]=df.iloc[:,5].astype(str)
    for idx,i in zip(df.index,df.iloc[:,5]):
        space = check_space(i)

        if space==2:
            unjoin1 = lambda m :str(m).split(' ')[0]
            unjoin2 = lambda m :str(m).split(' ')[1]
            unjoin3 = lambda m :str(m).split(' ')[2]
            new.loc[idx,'qty'] = unjoin1(i)
            new.loc[idx,'unit_listprice'] = unjoin2(i)
            new.loc[idx,'unit_discprice'] = unjoin3(i)
            # copy the rest of the values according to row indexes
            new.loc[idx,'list_pricetotal'] = df.iloc[idx,7]# 5th column
            new.loc[idx,'disc_pricetotal'] = df.iloc[idx,8]
            new.loc[idx,'po_num'] = po_num



        elif space==1:
            # remove error in qty where qty includes unit list price
            unjoin1 = lambda m :str(m).split(' ')[0]
            unjoin2 = lambda m :str(m).split(' ')[1]
            new.loc[idx,'qty'] = unjoin1(i)
            new.loc[idx,'unit_listprice'] = unjoin2(i)
            # copy the rest of the values according to row indexes
            new.loc[idx,'unit_discprice'] = df.iloc[idx,6]
            new.loc[idx,'list_pricetotal'] = df.iloc[idx,7]# 5th column
            new.loc[idx,'disc_pricetotal'] = df.iloc[idx,8]
            new.loc[idx,'po_num'] = po_num

        else:
            new.loc[idx,'qty'] = np.nan
    
    # drop row if qty is nan
    new.dropna(subset=['qty'], inplace=True)
    
    # convert null to today's date to allow datatime calculation
    new=new.replace(np.nan,'naan')
    

    new['exp_date']=[toMthYr(i) for i in new.batch]
    new['curr_date']=pd.to_datetime('today')
    new['num_months'] = month_diff(new.exp_date, new.curr_date)
    
    
    # convert 'exp_date' back to strings
    new['exp_date'] = [i.strftime("%Y-%m") for i in new['exp_date']]

    # reorganize columns of dataframe
    new = new[['material_no','material_desc','batch','qty','unit_listprice','unit_discprice',\
                                'list_pricetotal','disc_pricetotal','po_num','exp_date','num_months']]
    
    
    new['qty'] = [int(float(i)) for i in new['qty']]
    new['list_pricetotal']=new['list_pricetotal'].str.replace(',',' ').str.replace(' ','')
    new['list_pricetotal'] = [float(i) for i in new['list_pricetotal']]
    new['disc_pricetotal']=new['disc_pricetotal'].str.replace(',',' ').str.replace(' ','')
    new['disc_pricetotal'] = [float(i) for i in new['disc_pricetotal']]

    new['unit_listprice'] = [float(i) for i in new['unit_listprice']]
    new['unit_discprice'] = [float(i) for i in new['unit_discprice']]
    new.num_months = [int(i) for i in new.num_months]

    # combine new dataframe and others dataframe and assign to OA dataframe
    new = new.sort_values('material_no')
    new = new.reset_index(drop=True)
    
    # if batch number is "R" then num_months column should be "no match....."
    for i,b in zip(new.index, new.batch):
        if re.search('\-R\d{4}$',b):
            new.loc[i,'num_months']=0
    new['po_numT'] = [i[:4] for i in new.po_num]

    return new, others, po_num

In [6]:
# OA: groupby based on po numbers v = po_num, temp = consolidated OA
def execute2(temp):
    
    
    # slicing those products with at least 16 months shelve life
    temp_over16 = temp[temp.num_months >= 16]
    temp_under16 = temp[temp.num_months < 16]
    

    # reorganizating columns
    temp = temp[['material_no','material_desc','qty','unit_listprice','unit_discprice','list_pricetotal','disc_pricetotal',\
                 'po_num','po_numT','batch','exp_date','num_months']]
    
    # groupby product code and run based on each PO Number (file has multiple PO Number)
    tot_gby=pd.DataFrame()
    
    for i in temp.po_numT.unique():
        temp1 = temp[temp.po_numT==i]
        temp_gby = temp1.groupby(['material_no','material_desc','unit_listprice','unit_discprice','po_num','po_numT',
                     ]).agg(qty=('qty','sum'),list_pricetotal=('list_pricetotal','sum'), 
                                        disc_pricetotal=('disc_pricetotal','sum')).reset_index()
        tot_gby=pd.concat([tot_gby,temp_gby])
                
                
                
    non_promo_gby=pd.DataFrame(columns=tot_gby.columns)
    promo_gby = pd.DataFrame(columns=tot_gby.columns)
    
    tot_gby = tot_gby.reset_index(drop=True)
    _=tot_gby.copy()
    # create dataframe for non promo and promo products on loop (based on conditions)
    for c, l, d in zip(_.index,_.unit_listprice, _.unit_discprice):

        if d==0 and l==0 or d<l:
            promo_gby = promo_gby.append(_.loc[c])
        else:
            non_promo_gby = non_promo_gby.append(_.loc[c])

    return non_promo_gby, promo_gby, temp_under16, tot_gby


In [7]:
# remove all other rows besides those which has prinicpal item code
def pi_transform(pi):
    
    # remove unwanted strings before product codes
    def regsplit2(m):
        trim = lambda x: ' '.join(x.split())
        g = re.search('.*\-\d{3,4}[A-Z]',m) or re.search('^.*\d{6}',m) or re.search('^.*[A-Z]{2}\d{2}\-[A-Z]{2}',m)\
        or re.search('^.*[A-Z]\d{8}',m) or re.search('.*\-\d{3,4}',m)
        g = g.group()
        return trim(g)

    pi.columns = pi.columns.str.lower()
    pi.columns = pi.columns.str.replace(' ','_')
    pi=pi.astype(str)
    
    
    # create new dataframe for transformation
    pi_df_1=pd.DataFrame()
    
    # transform 
    pi['principal_item_code']=[' '.join(i.split()) for i in pi['principal_item_code']]
    pi['customer_purchase_order_#']= [' '.join(i.split()) for i in pi['customer_purchase_order_#']]
    
    pi['item_qty'] = [int(float(i)) for i in pi['item_qty']]
    pi['item_list_price']=pi['item_list_price'].str.replace(',',' ').str.replace(' ','')
    pi['item_list_price'] = [float(i) for i in pi['item_list_price']]
    pi['item_unit_price']=pi['item_unit_price'].str.replace(',',' ').str.replace(' ','')
    pi['item_unit_price'] = [float(i) for i in pi['item_unit_price']]
    
    # reorganize pi dataframe
    pi_df = pi[['principal_item_code','item_qty','item_list_price','item_unit_price',\
                'customer_purchase_order_#']]
    # replaceing black slash with _
    pi_df['customer_purchase_order_#']= [i.replace('/','_').replace('-','_')\
                                         for i in pi_df['customer_purchase_order_#']]
    

    # extract only rows with principal_item_code
    for idx, m in zip(pi_df.index,pi_df.principal_item_code):
        if re.search('.*\-\d{3,4}\D.*',m) or re.search('.*\-\d{3,4}\s',m)\
            or re.search('^\d{6}\s',m) or re.search('^\d{6}\D.*',m)\
            or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\D*',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}\s',m)\
            or re.search('^[A-Z]\d{8}\D.*',m) or re.search('^[A-Z]\d{8}\s.*',m) or re.search('.*\-\d{3,4}$',m)\
            or re.search('^\d{6}$',m) or re.search('^[A-Z]{2}\d{2}\-[A-Z]{2}$',m) or re.search('^[A-Z]\d{8}$',m):
            # concatentate each row
            pi_df_1 = pd.concat([pi_df_1, pi_df.loc[[idx]]])
            
    
    # correcting principal item code
    pi_df_1['principal_item_code']=[regsplit2(i) for i in pi_df_1['principal_item_code']]
    

    temp=pd.DataFrame()
    pi_df_1['customer_purchase_order_#'] = [' '.join(i.split()) for i in pi_df_1['customer_purchase_order_#']]
    
    for i in pi_df_1['customer_purchase_order_#'].unique():

        # groupby by customer purchase order
        p = pi_df_1[pi_df_1['customer_purchase_order_#']==i]
        p2 = p.groupby(['principal_item_code','item_list_price','item_unit_price','customer_purchase_order_#']).\
        agg(qty=('item_qty','sum'))
    
        temp = pd.concat([temp, p2])
        
    temp = temp.reset_index()
    
    # create another column for short form PO number
    temp['customer_purchase_order_#_T']=[i[:4] for i in temp['customer_purchase_order_#']]
    
    
    # tranform data type pi_gby
    temp.item_list_price=[float(i) for i in temp.item_list_price]
    temp.item_unit_price= [float(i) for i in temp.item_unit_price]
    temp.qty= [int(i) for i in temp.qty]
    
    
    non_promo_pi_df = pd.DataFrame()
    promo_pi_df = pd.DataFrame()
        
    # create dataframe for non promo and promo products on loop
    for c, l, d in zip(temp.index,temp.item_list_price, temp.item_unit_price):
        if d==0 and l==0 or d<l:
            promo_pi_df = promo_pi_df.append(temp.loc[c])
        else:
            non_promo_pi_df = non_promo_pi_df.append(temp.loc[c])
                
    
    return temp, non_promo_pi_df, promo_pi_df

In [8]:
# comparison between PI and OA (non promo)
def execute_np1(non_promo_pi_df,non_promo_gby):
    
#     oa_accu = pd.DataFrame(columns = ['material_no','unit_listprice','unit_discprice','po_numT'])
#     pi_accu = pd.DataFrame(columns = oa_accu.columns)

    # extract 4 columns
    non_promo_gby = non_promo_gby[['material_no','qty','unit_listprice','unit_discprice','po_numT']]
    non_promo_pi_df = non_promo_pi_df[['principal_item_code','qty','item_list_price','item_unit_price',\
                                       'customer_purchase_order_#_T']]
    
    
    # synchronize column names
    non_promo_pi_df.rename(columns={'principal_item_code':'material_no','item_list_price':'unit_listprice',
                                   'item_unit_price':'unit_discprice','customer_purchase_order_#_T':'po_numT'},inplace=True)
    
    non_promo_gby = non_promo_gby.reset_index(drop=True)
    non_promo_pi_df = non_promo_pi_df.reset_index(drop=True)
    
    pi_np = non_promo_pi_df.copy()
    oa_np = non_promo_gby.copy()
    
    pi_np = pi_np.sort_values(['material_no', 'po_numT'])
    pi_np = pi_np.reset_index(drop=True)
    oa_np = oa_np.sort_values(['material_no', 'po_numT'])
    oa_np = oa_np.reset_index(drop=True)
    
    diff = pd.concat([pi_np,oa_np]).drop_duplicates(keep=False)

    return diff, pi_np, oa_np
    
 

In [9]:
def execute_p1(promo_pi_df,promo_gby):
    
#     piInOa_accu = pd.DataFrame(columns = ['material_no','unit_listprice','unit_discprice','po_numT'])
#     piNotinOa_accu = pd.DataFrame(columns = piInOa_accu.columns)
#     oaNotinPi_accu = pd.DataFrame(columns = piInOa_accu.columns)
    
    # extract 4 columns
    promo_gby = promo_gby[['material_no','qty','unit_listprice','unit_discprice','po_numT']]
    promo_pi_df = promo_pi_df[['principal_item_code','qty','item_list_price','item_unit_price',\
                                       'customer_purchase_order_#_T']]
    
    
    # synchronize column names
    promo_pi_df.rename(columns={'principal_item_code':'material_no','item_list_price':'unit_listprice',
                                   'item_unit_price':'unit_discprice','customer_purchase_order_#_T':'po_numT'},inplace=True)
    
    promo_gby = promo_gby.reset_index(drop=True)
    promo_pi_df = promo_pi_df.reset_index(drop=True)
    
    
    pi_p = promo_pi_df.copy()
    oa_p = promo_gby.copy()
    
    
    pi_p = pi_p.sort_values(['material_no', 'po_numT'])
    pi_p = pi_p.reset_index(drop=True)
    oa_p = oa_p.sort_values(['material_no', 'po_numT'])
    oa_p = oa_p.reset_index(drop=True)
    
    diff = pd.concat([pi_p,oa_p]).drop_duplicates(keep=False)

    
    return diff, pi_p, oa_p
    
    

# PDF file input

In [10]:
# read excel sheet of input files and assign to df


# df = pd.read_excel('input_files2.xlsx')

# make list of OA and PI files
# df=df.fillna(0)
oa_files = []
pi_files = []
other_files = []
            
            
# iterate thru the files
for x in os.listdir(path):
    try:
        filename, extension = os.path.splitext(x)
#             print(filename, extension)
        if extension == '.pdf':
            oa_files.append(x)
        elif extension == '.csv':
            pi_files.append(x)
        else:
            other_files.append(x)
    except:
        continue
                
        
        
print("Converting OA PDF files to Excel Workbook. Please wait a few minutes......")
# concatenate OA files
attach=pd.DataFrame()

oa_po = pd.DataFrame(columns=['oa_filename','po_number'])

      
#counter
e=[] # empty list to contain po_num (which recurr)
h=0 # row number for OA and PI reference file
j=1 # counter for po_num file name (in case where there are recurring po_num)
for i in oa_files:
    df_ = read_pdf(path+i, pages='all', encoding='latin1')
    if len(df_)==0:
        with open(path_output+"Alert.txt","w") as b:
            b.write(f"{i} is corrupted. Please remove this OA and PI from data_input folder and run again....terminating sequence.")
        print(f"{i} is corrupted. Please remove this OA and PI from data_input folder and run again....terminating sequence.")
        sys.exit()
    else:
        new, others, po_num=execute(df_)
    
    # combine body and other info such as packing charges and del po_numT column
    converted = pd.concat([new,others])
    
    del converted['po_numT']

    # save fully converted OA record
    
    if po_num in e:
        po_num=po_num+'_'+str(j)
        converted.to_excel(path_output+f'1_ponum_{po_num}.xlsx', index=False)
        j+=1
    else:
        converted.to_excel(path_output+f'1_ponum_{po_num}.xlsx', index=False)
        e.append(po_num)
        
        
    # Create OA and PO reference file
    oa_po.loc[h,'oa_filename']=i
    oa_po.loc[h,'po_number']=po_num
    h+=1

    attach = pd.concat([attach,new])
    attach = attach.reset_index(drop=True)

# reference to indicate PO numbers belong to which OA filenames
oa_po.to_excel(path_output+'OA and PO Numbers.xlsx', index=False)


# execute2
# groupby OAs by  PO numbers

print("Consolidating OA files.......")

non_promo_gby, promo_gby, temp_under16, tot_gby = execute2(attach) # execution step

non_promo_gby.to_excel(path_output+'2_OA Non_Promo Consolidated file.xlsx',index=False)

promo_gby.to_excel(path_output+'2_OA Promo Consolidated file.xlsx',index=False)

temp_under16.to_excel(path_output+'2_OA under16 Consolidated file.xlsx',index=False)


Converting OA PDF files to Excel Workbook. Please wait a few minutes......
Consolidating OA files.......


In [13]:
print("Transforming PI files.......")
# transform PI files
non_promo_pi_df = pd.DataFrame(columns=['principal_item_code','item_list_price','item_unit_price',
                                       'customer_purchase_order_#','qty','customer_purchase_order_#_T'])

promo_pi_df = pd.DataFrame(columns=non_promo_pi_df.columns)
all_pi_df = pd.DataFrame(columns=non_promo_pi_df.columns)
for i in pi_files:
    pi = pd.read_csv(path+i, encoding='latin1')
    pi_df, non_promo_pi,promo_pi = pi_transform(pi)
    
    # concatentate non_promo and promo files
    non_promo_pi_df = pd.concat([non_promo_pi_df, non_promo_pi])
    promo_pi_df = pd.concat([promo_pi_df, promo_pi])
    all_pi_df = pd.concat([all_pi_df,pi_df])

# save non promo PI flles
non_promo_pi_df.to_excel(path_output+f"3_PI Non_Promo Consolidated.xlsx", index=False)

# save promo PI files
promo_pi_df.to_excel(path_output+f"3_PI Promo Consolidated.xlsx", index=False)

# save promo PI files
all_pi_df.to_excel(path_output+f"3_PI All Consolidated.xlsx", index=False)


print("Comparing PI and OA files........")


# Non Promo comparison between PI and OA #----------------------------------------------------

NP_diff, pi_np, oa_np  = execute_np1(non_promo_pi_df, non_promo_gby)

# overall diff between pi_np and oa_np. Reset index to ensure no duplicate index
NP_diff=NP_diff.reset_index(drop=True)

# extract items belonging to OA in NP_diff
x = pd.concat([NP_diff,oa_np])
dup_NP_diff_oa_np = x[x.duplicated(keep='first')]

# create new 'status' column
dup_NP_diff_oa_np['status']='Discrepancies in OA'
dup_NP_diff_oa_np=dup_NP_diff_oa_np.reset_index(drop=True)
    
# extract items belonging to PI in NP_diff
y = pd.concat([NP_diff,pi_np])
dup_NP_diff_pi_np = y[y.duplicated(keep='first')]

# These are in PI so, not discrepancy
dup_NP_diff_pi_np['status']='Correct items in PI'


NP_diff, pi_np, oa_np  = execute_np1(non_promo_pi_df, non_promo_gby)

# overall diff between pi_np and oa_np. Reset index to ensure no duplicate index
NP_diff=NP_diff.reset_index(drop=True)

# # extract items belonging to OA in NP_diff
# x = pd.concat([NP_diff,oa_np])
# dup_NP_diff_oa_np = x[x.duplicated(keep='first')]

# extract items belonging to OA in NP_diff
oa_npT = oa_np.drop_duplicates(keep='first')
x = pd.concat([NP_diff,oa_npT])
dup_NP_diff_oa_np = x[x.duplicated(keep='first')]

# create new 'status' column
dup_NP_diff_oa_np['status']='Discrepancies in OA'
dup_NP_diff_oa_np=dup_NP_diff_oa_np.reset_index(drop=True)

# extract items belonging to PI in NP_diff
pi_npT = pi_np.drop_duplicates(keep='first')
y = pd.concat([NP_diff,pi_npT])
dup_NP_diff_pi_np = y[y.duplicated(keep='first')]

# These are in PI so, not discrepancy
dup_NP_diff_pi_np['status']='Correct items in PI'

# extract missing code by concatenating both groups using only material no and po numT and changing status
j = pd.concat([dup_NP_diff_oa_np,dup_NP_diff_pi_np])
missingCode = j[~j.duplicated(subset=['material_no','po_numT'],keep=False)]
missingCode = missingCode.reset_index(drop=True)
for i,m in zip(missingCode.index,missingCode.status):
    missingCode.loc[i,'status']=missingtext(m)
    
tot = missingCode.copy()

# resulting status
NP_status = pd.concat([dup_NP_diff_pi_np,dup_NP_diff_oa_np,tot])
NP_status=NP_status.reset_index(drop=True)

# drop duplicate items due to missing codes repeating either in PI or OA
NP_status=NP_status.drop_duplicates(subset=['material_no','qty','unit_listprice','unit_discprice','po_numT'],keep='last')
NP_status.to_excel(path_output+'4_Non Promo reconciliation.xlsx', index=False)




# Promo comparison between PI and OA #----------------------------------------------------

P_diff, pi_p, oa_p = execute_p1(promo_pi_df, promo_gby)

# overall diff between pi_p and oa_p. Reset index to ensure no duplicate index
P_diff=P_diff.reset_index(drop=True)

# extract items belonging to OA in P_diff
oa_pT= oa_p.drop_duplicates(keep='first')
k=pd.concat([P_diff, oa_pT])
dup_P_diff_oa_p = k[k.duplicated(keep='first')]

# create new 'status' column
dup_P_diff_oa_p['status']='Discrepancies in OA'
dup_P_diff_oa_p=dup_P_diff_oa_p.reset_index(drop=True)

# extract items belonging to PI in P_diff
pi_pT = pi_p.drop_duplicates(keep='first')
v=pd.concat([P_diff, pi_pT])
dup_P_diff_pi_p = v[v.duplicated(keep='first')]
dup_P_diff_pi_p=dup_P_diff_pi_p.reset_index(drop=True)

# These are items in PI
dup_P_diff_pi_p['status']='Correct items in PI'


# extract missing code by concatenating both groups using only material no and po numT and changing status
j = pd.concat([dup_P_diff_oa_p,dup_P_diff_pi_p])
missingCode = j[~j.duplicated(subset=['material_no','po_numT'],keep=False)]
missingCode = missingCode.reset_index(drop=True)
for i,m in zip(missingCode.index,missingCode.status):
    missingCode.loc[i,'status']=missingtext(m)
    
tot = missingCode.copy()

# concatenate altogether
P_status = pd.concat([dup_P_diff_pi_p,dup_P_diff_oa_p,tot])
P_status=P_status.reset_index(drop=True)

# remove duplicates of missing code
P_status=P_status.drop_duplicates(subset=['material_no','qty','unit_listprice','unit_discprice','po_numT'],keep='last')
P_status.to_excel(path_output+'4_Promo reconciliation.xlsx', index=False)


# write to text file
with open(path_output+"summary.txt",'w') as t:
    t.write(f"Num of POs processed: {overallPO()}\n")
    t.write(f"{expiry()[2]} products under expiry from PO Num: {expiry()[0]}\n\n")
    t.write(f"Non promo discrepancies in OA for PO Num: {NPstatus()[0]}, Product Code: {NPstatus()[1]}\n")
    t.write(f"Non Promo missing codes in PO Num: {NPstatus()[3]}, Product Code: {NPstatus()[2]}\n\n")
    t.write(f"Promo discrepancies in OA for PO Num: {Pstatus()[0]}, Product Code: {Pstatus()[1]}\n")
    t.write(f"Promo missing codes in PO Num: {Pstatus()[3]}, Product Code: {Pstatus()[2]}\n")


print("Processing completed.......")



Transforming PI files.......
Comparing PI and OA files........
Processing completed.......
